# Data

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:

from backbone.utils.wfo_utils import run_strategy, run_wfo
from backbone.utils.general_purpose import calculate_units_size, diff_pips

import itertools
import numpy as np
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
from pandas import Timestamp
import numpy as np
import random
from backtesting import Strategy
from backtesting.lib import crossover
import talib as ta

random.seed(42)

In [3]:
INITIAL_CASH = 10_000
MARGIN = 1/30
COMMISSION = 7e-4

timezone = pytz.timezone("Etc/UTC")
date_from_get_data = datetime(2021, 10, 1, tzinfo=timezone)
date_to_get_data = datetime(2024, 9, 1, tzinfo=timezone)

limited_testing_start_date = Timestamp('2023-09-01 00:00:00', tz='UTC')
limited_testing_end_date = Timestamp('2024-09-01 00:00:00', tz='UTC')

In [4]:

groups = [
    'Forex_Indicator',
    'CryptoCross_grp',
    'Crypto_group',
    'Energies_group',
    'Forex_group',
    'Indices_group',
    'Stocks_group'
]

In [5]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

symbols = mt5.symbols_get()

tickers = [symbol.path.split('\\')[1] for symbol in symbols if ('Energies_group' in symbol.path or 'Indices_group' in symbol.path or 'Stocks_group' in symbol.path)]

# tickers = ['XPEVm', 'RLXm']

print(tickers)

intervals = [
    mt5.TIMEFRAME_H4,
    mt5.TIMEFRAME_H2,
    mt5.TIMEFRAME_H1,
    mt5.TIMEFRAME_M15,
]

symbols = get_data(tickers, intervals, date_from_get_data, date_to_get_data)

['UKOILm', 'USOILm', 'AUS200m', 'DE30m', 'FR40m', 'HK50m', 'JP225m', 'STOXX50m', 'UK100m', 'US30m', 'US500m', 'USTECm', 'AAPLm', 'ABBVm', 'ABTm', 'ADBEm', 'ADPm', 'AMDm', 'AMGNm', 'AMTm', 'AMZNm', 'ATVIm', 'AVGOm', 'BAm', 'BABAm', 'BACm', 'BIIBm', 'BMYm', 'Cm', 'CHTRm', 'CMCSAm', 'CMEm', 'COSTm', 'CSCOm', 'CSXm', 'CVSm', 'EAm', 'EBAYm', 'EQIXm', 'Fm', 'FBm', 'GILDm', 'GOOGLm', 'HDm', 'IBMm', 'INTCm', 'INTUm', 'ISRGm', 'JNJm', 'JPMm', 'KOm', 'LINm', 'LLYm', 'LMTm', 'MAm', 'MCDm', 'MDLZm', 'MMMm', 'MOm', 'MRKm', 'MSm', 'MSFTm', 'NFLXm', 'NKEm', 'NVDAm', 'ORCLm', 'PEPm', 'PFEm', 'PGm', 'PMm', 'PYPLm', 'REGNm', 'SBUXm', 'Tm', 'TMOm', 'TMUSm', 'TSLAm', 'UNHm', 'UPSm', 'Vm', 'VRTXm', 'VZm', 'WFCm', 'WMTm', 'XOMm', 'AMCm', 'BBm', 'BBBYm', 'BEKEm', 'BIDUm', 'BILIm', 'BRQSm', 'BYNDm', 'CANm', 'EDUm', 'FTNTm', 'FUTUm', 'IQm', 'JDm', 'LIm', 'NIOm', 'NTESm', 'PDDm', 'RLXm', 'TALm', 'TIGRm', 'TMEm', 'TSMm', 'VIPSm', 'XPEVm', 'YUMCm', 'ZTOm', 'XNGUSDm', 'METAm', 'IN50m']
MetaTrader5 package author: 

In [6]:
max_start_date = None
intervals_start_dates = {}

tickers = np.unique(list(symbols.keys())).tolist()

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date


intervals_start_dates


print(intervals_start_dates)
print(limited_testing_start_date)
print(limited_testing_end_date)

{16388: Timestamp('2022-03-20 20:00:00+0000', tz='UTC'), 16386: Timestamp('2022-03-20 22:00:00+0000', tz='UTC'), 16385: Timestamp('2022-03-20 22:00:00+0000', tz='UTC'), 15: Timestamp('2022-03-20 22:15:00+0000', tz='UTC')}
2023-09-01 00:00:00+00:00
2024-09-01 00:00:00+00:00


# Estrategia

In [7]:
class Macd(Strategy):
    pip_value = None
    minimum_units = None
    maximum_units = None
    contract_volume = None
    opt_params = None
    
    risk=1
    atr_multiplier = 1.5
    sma_period = 200
    macd_fast_period = 7
    macd_slow_period = 26
    macd_signal_period = 9
    
    cum_rsi_open_threshold = 65
    cum_rsi_close_threshold = 45
    
    rsi_period = 2

    macd = pd.Series(dtype=float)
    macdsignal = pd.Series(dtype=float)
    macdhist = pd.Series(dtype=float)

    def init(self):
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )
        
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)
        self.rsi = self.I(ta.RSI, self.data.Close, timeperiod=self.rsi_period)
        
        
    def next(self):
        
        actual_date = self.data.index[-1]
        
        if self.opt_params and actual_date in self.opt_params.keys():
            for k, v in self.opt_params[actual_date].items():
                setattr(self, k, v)
        
            self.macd_slow_period = int(2.17 * self.macd_signal_period)
            self.macd_fast_period = int(1.33 * self.macd_signal_period)
                        
        self.macd, self.macdsignal, self.macdhist = ta.MACD(
            self.data.Close, 
            fastperiod=self.macd_fast_period, 
            slowperiod=self.macd_slow_period, 
            signalperiod=self.macd_signal_period
        )

        actual_close = self.data.Close[-1]
        cum_rsi = self.rsi[-1] + self.rsi[-2]
    
        if self.position:
            if self.position.is_long:
                if cum_rsi > self.cum_rsi_close_threshold:
                    self.position.close()

            if self.position.is_short:
                if cum_rsi < 100 - self.cum_rsi_close_threshold:
                    self.position.close()

        else:

            if crossover(self.macdsignal, self.macd) and cum_rsi <= 100 - self.cum_rsi_open_threshold and actual_close > self.sma[-1]:        
                sl_price = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value,
                    maximum_units=self.maximum_units,
                    minimum_units=self.minimum_units
                )
                
                self.buy(
                    size=units,
                    sl=sl_price
                )
                
            if crossover(self.macd, self.macdsignal) and cum_rsi >= self.cum_rsi_open_threshold and actual_close < self.sma[-1]:
                sl_price = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value,
                    maximum_units=self.maximum_units,
                    minimum_units=self.minimum_units
                )
                
                self.sell(
                    size=units,
                    sl=sl_price
                )
                
STRATEGY = Macd

# Analisis preliminar

In [8]:
strategies = [
    STRATEGY
]

experiments = list(itertools.product(
    tickers, intervals, strategies
))

stats_per_symbol = {}

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:
    try:
        print(ticker, interval)
        
        if ticker not in stats_per_symbol.keys():
            stats_per_symbol[ticker] = {}
        
        prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date]

        df_stats, stats = run_strategy(
            strategy=strategy,
            ticker=ticker,
            interval=interval,
            commission=COMMISSION, 
            prices=prices, 
            initial_cash=INITIAL_CASH, 
            margin=MARGIN, 
            plot=False
        )

        performance = pd.concat([performance, df_stats])
        stats_per_symbol[ticker][interval] = stats

    except Exception as e:
        print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


AAPLm 16388
AAPLm 16386
AAPLm 16385
AAPLm 15
ABBVm 16388
ABBVm 16386
ABBVm 16385
ABBVm 15
ABTm 16388
ABTm 16386
ABTm 16385
ABTm 15
ADBEm 16388
ADBEm 16386
ADBEm 16385
ADBEm 15
ADPm 16388
ADPm 16386
ADPm 16385
ADPm 15
AMDm 16388
AMDm 16386
AMDm 16385
AMDm 15
AMGNm 16388
AMGNm 16386
AMGNm 16385
AMGNm 15
AMTm 16388
AMTm 16386
AMTm 16385
AMTm 15
AMZNm 16388
AMZNm 16386
AMZNm 16385
AMZNm 15
AUS200m 16388
AUS200m 16386
AUS200m 16385
AUS200m 15
AVGOm 16388
AVGOm 16386
AVGOm 16385
AVGOm 15
BABAm 16388
BABAm 16386
BABAm 16385
BABAm 15
BACm 16388
BACm 16386
BACm 16385
BACm 15
BAm 16388
BAm 16386
BAm 16385
BAm 15
BBm 16388
BBm 16386
BBm 16385
BBm 15
BEKEm 16388
BEKEm 16386
BEKEm 16385
BEKEm 15
BIDUm 16388
BIDUm 16386
BIDUm 16385
BIDUm 15
BIIBm 16388
BIIBm 16386
BIIBm 16385
BIIBm 15
BILIm 16388
BILIm 16386
BILIm 16385
BILIm 15
BMYm 16388
BMYm 16386
BMYm 16385
BMYm 15
CHTRm 16388
CHTRm 16386
CHTRm 16385
CHTRm 15
CMCSAm 16388
CMCSAm 16386
CMCSAm 16385
CMCSAm 15
CMEm 16388
CMEm 16386
CMEm 16385
CMEm 

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:84: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:84: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(


JP225m 16385


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:84: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(


JP225m 15


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backbone\utils\wfo_utils.py:84: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_train = Backtest(


JPMm 16388
JPMm 16386
JPMm 16385
JPMm 15
KOm 16388
KOm 16386
KOm 16385
KOm 15
LINm 16388
LINm 16386
LINm 16385
LINm 15
LIm 16388
LIm 16386
LIm 16385
LIm 15
LLYm 16388
LLYm 16386
LLYm 16385
LLYm 15
LMTm 16388
LMTm 16386
LMTm 16385
LMTm 15
MAm 16388
MAm 16386
MAm 16385
MAm 15
MCDm 16388
MCDm 16386
MCDm 16385
MCDm 15
MDLZm 16388
MDLZm 16386
MDLZm 16385
MDLZm 15
METAm 16388
METAm 16386
METAm 16385
METAm 15
MMMm 16388
MMMm 16386
MMMm 16385
MMMm 15
MOm 16388
MOm 16386
MOm 16385
MOm 15
MRKm 16388
MRKm 16386
MRKm 16385
MRKm 15
MSFTm 16388
MSFTm 16386
MSFTm 16385
MSFTm 15
MSm 16388
MSm 16386
MSm 16385
MSm 15
NFLXm 16388
NFLXm 16386
NFLXm 16385
NFLXm 15
NIOm 16388
NIOm 16386
NIOm 16385
NIOm 15
NKEm 16388
NKEm 16386
NKEm 16385
NKEm 15
NTESm 16388
NTESm 16386
NTESm 16385
NTESm 15
NVDAm 16388
NVDAm 16386
NVDAm 16385
NVDAm 15
ORCLm 16388
ORCLm 16386
ORCLm 16385
ORCLm 15
PDDm 16388
PDDm 16386
PDDm 16385
PDDm 15
PEPm 16388
PEPm 16386
PEPm 16385
PEPm 15
PFEm 16388
PFEm 16386
PFEm 16385
PFEm 15
PGm 1638

In [9]:
0/0

ZeroDivisionError: division by zero

In [13]:
filter_performance = performance[
    (performance['return/dd']>2)
    & (performance['stability_ratio']>0.7)
    & (performance['trades']>30)
].sort_values(by=['custom_metric'], ascending=[False]).drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,Macd,PFEm,15,0.738509,35.015481,13501.548070,9.386681,148 days 23:00:00,57.068063,0.799399,191,0.075223,15.093223,13501.548070,364 days 09:15:00,3.730337,17.724019
0,Macd,CVSm,15,0.713631,42.487172,14248.717160,14.000496,116 days 01:30:00,52.903226,0.540457,155,0.085611,13.122100,14248.717160,364 days 06:00:00,3.034691,14.303134
0,Macd,ZTOm,16385,0.904878,15.235744,11523.574381,3.266862,79 days 18:00:00,63.265306,1.515691,49,0.318797,14.114286,11523.574381,364 days 06:00:00,4.663724,13.968716
0,Macd,NTESm,16385,0.864253,24.714919,12471.491878,6.613884,81 days 00:00:00,58.208955,0.892618,67,0.435687,15.264798,12471.491878,364 days 08:00:00,3.736824,13.696662
0,Macd,XNGUSDm,16388,0.785379,19.731556,11973.155576,4.753635,125 days 04:00:00,64.583333,1.866950,48,0.907683,14.196373,11973.155576,364 days 20:00:00,4.150835,13.346635
0,Macd,XPEVm,15,0.800622,21.090193,12109.019292,8.010957,125 days 19:45:00,57.510730,1.071225,233,0.094940,14.587548,12109.019292,364 days 09:15:00,2.632668,12.768208
0,Macd,Cm,16385,0.853345,7.618863,10761.886350,2.818787,130 days 03:00:00,62.500000,1.093356,48,0.077227,8.526188,10761.886350,364 days 09:00:00,2.702888,7.764573
0,Macd,CMEm,16385,0.756244,10.377820,11037.781958,4.862216,74 days 02:00:00,65.116279,1.294897,43,0.166718,15.371429,11037.781958,364 days 06:00:00,2.134381,6.699112
0,Macd,CSCOm,16385,0.803766,8.516994,10851.699413,4.095839,76 days 23:00:00,64.444444,1.128145,45,0.141894,13.771429,10851.699413,364 days 06:00:00,2.079426,6.399048


In [14]:
for ticker, interval in zip(portfolio, intervals):
    
    prices = symbols[ticker][interval].loc[limited_testing_start_date:limited_testing_end_date]

    df_stats = run_strategy(
        strategy=STRATEGY,
        ticker=ticker,
        interval=interval,
        commission=COMMISSION, 
        prices=prices, 
        initial_cash=INITIAL_CASH, 
        margin=MARGIN,
        plot=True
    )


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarn

# WFO

In [16]:
from backbone.utils.wfo_utils import optimization_function

lookback_bars = 2000
validation_bars = 200
warmup_bars = 200

all_wfo_performances = pd.DataFrame()
all_opt_params = {}

optimized_params = {
    'macd_signal_period' : [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'atr_multiplier' : [1.5, 1.8, 2, 2.5, 1.8, 3],
}

for index, row in filter_performance.iterrows():
    
    try:
        ticker = row.ticker
        interval = row.interval
        
        print(ticker, interval)
        
        prices = symbols[ticker][interval]

        wfo_stats, df_stats, opt_params = run_wfo(
            strategy=STRATEGY,
            ticker=ticker,
            interval=interval,
            prices=prices,
            initial_cash=INITIAL_CASH,
            commission=COMMISSION,
            margin=MARGIN,
            optim_func=optimization_function,
            params=optimized_params,
            lookback_bars=lookback_bars,
            warmup_bars=warmup_bars,
            validation_bars=validation_bars,
            plot=False
        )
                
        if ticker not in all_opt_params.keys():
            all_opt_params[ticker] = {}

        all_opt_params[ticker][interval] = opt_params

        all_wfo_performances = pd.concat([all_wfo_performances, df_stats])
    
    except Exception as e:
        print(f'No se pudo ejecutar para el ticker {ticker}: {e}')
    
all_wfo_performances['return/dd'] = all_wfo_performances['return'] / -all_wfo_performances['drawdown']
all_wfo_performances['drawdown'] = -all_wfo_performances['drawdown']
all_wfo_performances['custom_metric'] = (all_wfo_performances['return'] / (1 + all_wfo_performances.drawdown)) * np.log(1 + all_wfo_performances.trades)

all_wfo_performances.drawdown_duration = pd.to_timedelta(all_wfo_performances.drawdown_duration)
all_wfo_performances.drawdown_duration = all_wfo_performances.drawdown_duration.dt.days

PFEm 15
CVSm 15
ZTOm 16385
NTESm 16385
XNGUSDm 16388
XPEVm 15
Cm 16385
CMEm 16385
CSCOm 16385


In [19]:
performance = pd.DataFrame()

wfo_stats_per_symbol = {}

for index, row in all_wfo_performances.iterrows():
    try:
        ticker = row.ticker
        interval = row.interval
        print(ticker, interval)
        
        prices = symbols[ticker][interval]
        optimized_params = all_opt_params[ticker][interval]
        
        if ticker not in wfo_stats_per_symbol.keys():
            wfo_stats_per_symbol[ticker] = {}

        df_stats, wfo_stats = run_strategy(
            strategy=STRATEGY,
            ticker=ticker,
            interval=interval,
            commission=COMMISSION, 
            prices=prices, 
            initial_cash=INITIAL_CASH, 
            margin=MARGIN, 
            opt_params=optimized_params, 
            plot=True
        )

        wfo_stats_per_symbol[ticker][interval] = wfo_stats

        performance = pd.concat([performance, df_stats])

    except Exception as e:
        print(f'hubo un problema con {ticker} {interval}: {e}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)

PFEm 15


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


CVSm 15


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


ZTOm 16385


...

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


XNGUSDm 16388


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


XPEVm 15


KeyboardInterrupt: 

In [18]:
filtered_wfo_performance = performance[
    # (performance['stability_ratio'] > 0.6)
    (performance['return/dd'] > 2)
].sort_values(by='return/dd', ascending=False)

filtered_wfo_performance

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,Macd,XNGUSDm,16388,0.674831,12.877033,11287.703293,6.14083,261 days 12:00:00,59.615385,0.689799,104,0.300474,13.761468,11287.703293,894 days,2.096953,8.39247


# Montecarlo

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from backbone.utils.montecarlo_utils import montecarlo_statistics_simulation

# Crear una lista para almacenar los resultados de cada ticker
data_drawdown = []
data_return = []
montecarlo_simulations = {}

all_drawdowns = pd.DataFrame()
all_returns = pd.DataFrame()

for index, row in filtered_wfo_performance.iterrows():
    ticker = row.ticker
    interval = row.interval
    
    print(f"Procesando ticker: {ticker}")
    trades_history = stats_per_symbol[ticker][interval]._trades
    eq_curve = stats_per_symbol[ticker][interval]._equity_curve
    
    # Simulación de Montecarlo para cada ticker (datos agregados)
    mc, synthetic_drawdown_curve, synthetic_return_curve = montecarlo_statistics_simulation(
        equity_curve=eq_curve,
        trade_history=trades_history, 
        n_simulations=100_000, 
        initial_equity=INITIAL_CASH, 
        threshold_ruin=0.8, 
        return_raw_curves=True,
        percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
    )
    
    montecarlo_simulations[ticker] = mc
    
    synthetic_drawdown_curve['ticker'] = ticker
    synthetic_return_curve['ticker'] = ticker
    
    all_drawdowns = pd.concat([all_drawdowns, synthetic_drawdown_curve])
    all_returns = pd.concat([all_returns, synthetic_return_curve])

In [ ]:
dd_df = pd.DataFrame()
returns_df = pd.DataFrame()

for ticker, mc in montecarlo_simulations.items():
    mc = mc.rename(
        columns={
            'Drawdown (%)': f'drawdown_{ticker}',
            'Final Return (%)': f'return_{ticker}',
        }
    )

    if dd_df.empty:
        dd_df = mc[[f'drawdown_{ticker}']]
    
    else:
        dd_df = pd.merge(
            dd_df,
            mc[[f'drawdown_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
    if returns_df.empty:
        returns_df = mc[[f'return_{ticker}']]
    
    else:
        returns_df = pd.merge(
            returns_df,
            mc[[f'return_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
display(dd_df)
display(returns_df)


In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(25, 18))
sns.boxplot(data=all_drawdowns, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = all_drawdowns["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = all_drawdowns["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()


In [ ]:
# Generar el gráfico
plt.figure(figsize=(12, 20))
sns.boxplot(data=all_returns, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = all_returns["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = all_returns["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()
